In [1]:
# running on summit nublado

# Run PTC Script on Notebook

Notebook adapted from the `scripts/`


In [2]:
import os, sys, time, datetime, glob, subprocess, argparse
def runCommand(cmd):
    # Run a command with subprocess
    print('Running command: \n')
    print(cmd)
    sys.stdout.flush()
    thisStep = subprocess.Popen(cmd, shell=True)
    subprocess.Popen.wait(thisStep)
    return

## Define variables

In [25]:
summit = True
obsDate=20220922

bias0=32
bias1=40

dark0=52
dark1=62

# darks for bright defects
defect0=55
defect1=57

# flats for dark defects
defect2=102
defect3=103

# sef of flats w/ same exposure (Empty)
# flat0=$(printf "%05d" None)
# flat1=$(printf "%05d" None)

# sef of flats w/ increasing exposure to the PTC curve (Empty)
flat_ptc0=72
flat_ptc1=103


In [19]:
if summit:
    repo = '/repo/LATISS/butler.yaml'
else:
    repo = '/repo/main/butler.yaml'
    
print(f'you are running on repo: {repo[:-12]}')


you are running on repo: /repo/LATISS


## Start Code


In [4]:
print("Starting DM Code")
print(f"Observation date is: {obsDate}")
print(" ")

Starting DM Code
Observation date is: 20220922
 


### Master Bias

In [7]:
print("Creating Master Bias")
print(f"pipetask run -j 8 -d detector IN (0) AND exposure IN ({obsDate}{bias0:05d}..{obsDate}{bias1:05d}) AND instrument='LATISS'")

cmd = f"""pipetask run -j 8 -d "detector IN (0) AND exposure IN ({obsDate}{bias0:05d}..{obsDate}{bias1:05d}) \
	 AND instrument='LATISS' " \
	 -b {repo} -i LATISS/raw/all,LATISS/calib \
	 -o u/jesteves/latiss/bias_{obsDate} \
	 -p $CP_PIPE_DIR/pipelines/Latiss/cpBias.yaml --register-dataset-types"""


Creating Master Bias
pipetask run -j 8 -d detector IN (0) AND exposure IN (2022092200032..2022092200040) AND instrument='LATISS'


In [8]:
runCommand(cmd)

Running command: 

pipetask run -j 8 -d "detector IN (0) AND exposure IN (2022092200032..2022092200040) 	 AND instrument='LATISS' " 	 -b /repo/LATISS/butler.yaml -i LATISS/raw/all,LATISS/calib 	 -o u/jesteves/latiss/bias_20220922 	 -p $CP_PIPE_DIR/pipelines/Latiss/cpBias.yaml --register-dataset-types


lsst.ctrl.mpexec.cmdLineFwk INFO: QuantumGraph contains 10 quanta for 2 tasks, graph ID: '1664317069.2572496-9525'
lsst.obs.lsst.assembly WARNING: /data/lsstdata/base/auxtel/oods/gen3butler/raw/20220922/000036/AT_O_20220922_000036_R00_S00.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
lsst.obs.lsst.assembly WARNING: /data/lsstdata/base/auxtel/oods/gen3butler/raw/20220922/000032/AT_O_20220922_000032_R00_S00.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
lsst.obs.lsst.assembly WARNING: /data/lsstdata/base/auxtel/oods/gen3butler/raw/20220922/000033/AT_O_20220922_000033_R00_S00.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
lsst.obs.lsst.assembly WARNING: /data/lsstdata/base/auxtel/oods/gen3butler/raw/20220922/000034/AT_O_20220

In [20]:
# Ingest the bias calibs:
cmd = f"""butler certify-calibrations {repo[:-12]} u/jesteves/latiss/bias_{obsDate} \
       u/jesteves/calib/latiss/bias_{obsDate} --begin-date 1980-01-01 \
       --end-date 2050-01-01 bias"""

runCommand(cmd)

Running command: 

butler certify-calibrations /repo/LATISS u/jesteves/latiss/bias_20220922        u/jesteves/calib/latiss/bias_20220922 --begin-date 1980-01-01        --end-date 2050-01-01 bias


Things worked fine

### Master Dark

In [26]:
print("Creating Master Dark")

# Build the dark calibs:
cmd = f"""pipetask run -j 8 -d "detector IN (0) AND exposure IN ({obsDate}{dark0:05d}..{obsDate}{dark1:05d}) \
	 AND instrument='LATISS' " \
	 -b {repo} -i LATISS/raw/all,LATISS/calib \
	 -o u/jesteves/latiss/dark_{obsDate} \
	 -p $CP_PIPE_DIR/pipelines/Latiss/cpDark.yaml \
	 -c isr:doDefect=False --register-dataset-types"""

runCommand(cmd)

Creating Master Dark
Running command: 

pipetask run -j 8 -d "detector IN (0) AND exposure IN (2022092200052..2022092200062) 	 AND instrument='LATISS' " 	 -b /repo/LATISS/butler.yaml -i LATISS/raw/all,LATISS/calib 	 -o u/jesteves/latiss/dark_20220922 	 -p $CP_PIPE_DIR/pipelines/Latiss/cpDark.yaml 	 -c isr:doDefect=False --register-dataset-types


lsst.ctrl.mpexec.cmdLineFwk INFO: QuantumGraph contains 23 quanta for 3 tasks, graph ID: '1664317570.3455186-10079'
lsst.obs.lsst.assembly WARNING: /data/lsstdata/base/auxtel/oods/gen3butler/raw/20220922/000060/AT_O_20220922_000060_R00_S00.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
lsst.obs.lsst.assembly WARNING: /data/lsstdata/base/auxtel/oods/gen3butler/raw/20220922/000059/AT_O_20220922_000059_R00_S00.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
lsst.obs.lsst.assembly WARNING: /data/lsstdata/base/auxtel/oods/gen3butler/raw/20220922/000056/AT_O_20220922_000056_R00_S00.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
lsst.obs.lsst.assembly WARNING: /data/lsstdata/base/auxtel/oods/gen3butler/raw/20220922/000058/AT_O_2022

In [28]:
# # Ingest the dark calibs:
cmd = f"""butler certify-calibrations {repo[:-12]} u/jesteves/latiss/dark_{obsDate}\
       u/jesteves/calib/latiss/dark_{obsDate} --begin-date 1980-01-01 \
       --end-date 2050-01-01 dark"""
runCommand(cmd)

Running command: 

butler certify-calibrations /repo/LATISS u/jesteves/latiss/dark_20220922       u/jesteves/calib/latiss/dark_20220922 --begin-date 1980-01-01        --end-date 2050-01-01 dark


lsst.daf.butler.cli.utils ERROR: Caught an exception, details are in traceback:
Traceback (most recent call last):
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/daf_butler/g1a6169c876+17c7ad0751/python/lsst/daf/butler/cli/cmd/commands.py", line 548, in certify_calibrations
    script.certifyCalibrations(*args, **kwargs)
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/daf_butler/g1a6169c876+17c7ad0751/python/lsst/daf/butler/script/certifyCalibrations.py", line 70, in certifyCalibrations
    raise RuntimeError(f"No inputs found for dataset {dataset_type_name} in {input_collection}.")
RuntimeError: No inputs found for dataset dark in u/jesteves/latiss/dark_20220922/20220927T222613Z.


In [33]:
##################### To Translate to Python #####################

# # Start the chained collection:
# butler collection-chain /repo/main u/jesteves/calib/latiss/calib.${obsDate} \
#        u/jesteves/calib/latiss/bias_${obsDate} \
#        u/jesteves/calib/latiss/dark_${obsDate} 

# echo "Creating Defects"
# # Build the defect calibs:
# pipetask run -j 8 -d "detector IN (0) AND exposure IN \
# 	 (${obsDate}${defect0}..${obsDate}${defect1}, ${obsDate}${defect2}..${obsDate}${defect3}) \
# 	 AND instrument='LATISS' " \
# 	 -b /repo/main/butler.yaml \
# 	 -i LATISS/raw/all,LATISS/calib,u/jesteves/calib/latiss/calib.${obsDate} \
# 	 -o u/jesteves/latiss/defects_${obsDate} \
# 	 -p $CP_PIPE_DIR/pipelines/Latiss/findDefects.yaml \
# 	 --register-dataset-types

# # Ingest the defect calibs:
# butler certify-calibrations /repo/main u/jesteves/latiss/defects_${obsDate} \
#        u/jesteves/calib/latiss/defects_${obsDate} --begin-date 1980-01-01 \
#        --end-date 2050-01-01 defects

# # Add the defects to the chained collection:

# butler collection-chain /repo/main --mode=extend \
#        u/jesteves/calib/latiss/calib.${obsDate} \
#        u/jesteves/calib/latiss/defects_${obsDate}

# echo "Creating Master Flat"
# # Build the flat calibs:
# pipetask run -j 8 -d "detector IN (0) AND exposure IN (${obsDate}${flat0}..${obsDate}${flat1}) \
# 	 AND instrument='LATISS' " \
# 	 -b /repo/main/butler.yaml \
# 	 -i LATISS/raw/all,LATISS/calib,u/jesteves/calib/latiss/calib.${obsDate} \
# 	 -o u/jesteves/latiss/flat_${obsDate} \
# 	 -p $CP_PIPE_DIR/pipelines/Latiss/cpFlat.yaml --register-dataset-types

# # Ingest the flat calibs:

# butler certify-calibrations /repo/main u/jesteves/latiss/flat_${obsDate} \
#        u/jesteves/calib/latiss/flat_${obsDate} --begin-date 1980-01-01 \
#        --end-date 2050-01-01 flat

# # Add the flat to the chained collection:

# butler collection-chain /repo/main --mode=extend \
#        u/jesteves/calib/latiss/calib.${obsDate} \
#        u/jesteves/calib/latiss/flat_${obsDate} 

# echo "Creating PTC Curves"
# echo "exposure IN (${obsDate}${flat_ptc0}..${obsDate}${flat_ptc1}) AND exposure.observation_type='flat'"

# # Run the PTC: 
# pipetask run -j 32 -d "detector IN (0) AND instrument='LATISS' AND \
# 	 exposure IN (${obsDate}${flat_ptc0}..${obsDate}${flat_ptc1}) AND exposure.observation_type='flat'" \
# 	 -b /repo/main \
#      -c isr:doFlat=False \
#      -i LATISS/raw/all,LATISS/calib,LATISS/calib,u/jesteves/calib/latiss/calib.${obsDate} \
# 	 -o u/jesteves/latiss/ptc_${obsDate}_flat \
# 	 -p $CP_PIPE_DIR/pipelines/Latiss/cpPtc.yaml \
# 	 --register-dataset-types

# echo "PTC collection: u/jesteves/latiss/ptc_${obsDate}" 
# echo "Congrats ! It's Done"

# Plot the PTC:

# plotPhotonTransferCurve.py \ /repo/main/u/jesteves/latiss/ptc_20220505/20220506T011002Z/ptc/ptc_LATISS_RXX_S00_u_jesteves_latiss_ptc_20220505_20220506T011002Z.fits \
# --detNum=0 --outDir=./
# plotPhotonTransferCurve.py /repo/main/u/jesteves/latiss/ptc_20220505_SDSSi/20220506T014156Z/ptc/ptc_LATISS_RXX_S00_u_jesteves_latiss_ptc_20220505_SDSSi_20220506T014156Z.fits --detNum=0 --outDir=./

# plotPhotonTransferCurve.py /repo/main/u/jesteves/latiss/ptc_20220405/20220408T211458Z/ptc/ptc_LATISS_RXX_S00_u_jesteves_latiss_ptc_20220405_20220408T211458Z.fits --detNum=0 --outDir=./

# ## look for exposures
# obsDate=20220504
# butler query-dimension-records /repo/main exposure --where "instrument='LATISS' AND exposure.observation_type='flat' AND exposure.day_obs > $((obsDate-1)) AND exposure.day_obs <$((obsDate+1))"

# Check Files on Butles

In [22]:
!ls $CP_PIPE_DIR/pipelines/Latiss/

cpBfkSolve.yaml  cpFringe.yaml		findDefectsPostFlat.yaml
cpBias.yaml	 cpLinearitySolve.yaml	findDefects.yaml
cpDark.yaml	 cpPtc.yaml		measurePtcWithLinearity.yaml
cpFlat.yaml	 cpSky.yaml


In [30]:
# Gen3 butler
# from lsst.daf.butler import Butler
import lsst.daf.butler as dafButler

dayObs = '2022-03-17'
dayObs = int(dayObs.replace('-', ''))

# Instantiate the Gen3 butler
dataPath = "/repo/main"
instrument = "LATISS"
obsDate = 20210708
butler = dafButler.Butler(dataPath, 
                          collections=["LATISS/raw/all", "LATISS/calib/unbounded",f"u/jesteves/calib/latiss/bias_{obsDate}"],
                          instrument=instrument)
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    if record.observation_type=='flat':
        print(record.id, record.observation_type, record.exposure_time, record.physical_filter, record.target_name)

2022031700014 flat 10.0 SDSSg~holo4_003 FlatField position
2022031700015 flat 2.0 SDSSg~holo4_003 FlatField position
2022031700016 flat 2.0 SDSSg~holo4_003 FlatField position
2022031700022 flat 20.0 SDSSg~holo4_003 FlatField position
2022031700023 flat 2.0 SDSSg~holo4_003 FlatField position
2022031700104 flat 2.0 SDSSr~empty FlatField position
2022031700105 flat 2.0 SDSSr~empty FlatField position
2022031700106 flat 2.0 SDSSr~empty FlatField position
2022031700107 flat 2.0 SDSSr~empty FlatField position
2022031700108 flat 2.0 SDSSr~empty FlatField position
2022031700109 flat 2.0 SDSSr~empty FlatField position
2022031700110 flat 2.0 SDSSr~empty FlatField position
2022031700111 flat 2.0 SDSSr~empty FlatField position
2022031700112 flat 2.0 SDSSr~empty FlatField position
2022031700113 flat 2.0 SDSSr~empty FlatField position
2022031700114 flat 2.0 SDSSr~empty FlatField position
2022031700115 flat 2.0 SDSSr~empty FlatField position
2022031700116 flat 2.0 SDSSr~empty FlatField position
202203